In [7]:
import pandas as pd
import numpy as np

from datetime import timedelta

from TAcharts.utils import group_candles

In [48]:
eth_1h = pd.read_csv('../data/ETH.csv')
eth_1h.head()

,date,open,high,low,close,volume
0,2019-01-01 00:00:00,132.74,133.50,132.67,133.36,6204.28401
1,2019-01-01 01:00:00,133.36,134.30,133.01,133.56,10214.95098
2,2019-01-01 02:00:00,133.56,135.30,133.19,134.52,16342.20664
3,2019-01-01 03:00:00,134.55,134.78,133.55,133.70,14895.35710
4,2019-01-01 04:00:00,133.72,134.14,133.66,133.86,6388.10191


In [49]:
# Group ETH into 1d candles
eth_1d = group_candles(eth_1h, interval=24)
eth_1d

,date,open,high,low,close,volume
0,2019-01-01 00:00:00,132.74,140.67,131.00,138.99,4.151021e+05
1,2019-01-02 00:00:00,138.91,155.37,138.30,151.93,1.018141e+06
2,2019-01-03 00:00:00,151.78,154.20,143.11,153.33,7.381434e+05
3,2019-01-04 00:00:00,153.11,159.26,144.66,154.51,8.914434e+05
4,2019-01-05 00:00:00,157.70,157.80,147.51,150.47,6.645218e+05
...,...,...,...,...,...,...
325,2019-11-23 02:00:00,150.00,154.33,146.60,150.02,3.124310e+05
326,2019-11-24 02:00:00,149.34,151.20,131.45,143.69,6.116189e+05
327,2019-11-25 04:00:00,137.37,151.50,136.79,146.76,5.985306e+05
328,2019-11-26 04:00:00,146.77,148.72,140.84,142.72,3.943000e+05


## Determine Rebalance Dates
### Daily Volatility

In [50]:
def calc_pct_change(high, low, _open):
    return (high - low) / _open

In [51]:
eth_1d['pct_change'] = calc_pct_change(eth_1d['high'], eth_1d['low'], eth_1d['open'])
eth_1d['weekday'] = pd.DatetimeIndex(eth_1d['date']).day_name()

daily_volatility = eth_1d.groupby('weekday')['pct_change'].mean().sort_values(ascending=False)
print(daily_volatility)

weekday
Tuesday      0.070740
Wednesday    0.066079
Sunday       0.063304
Monday       0.062416
Friday       0.061621
Thursday     0.059621
Saturday     0.052946
Name: pct_change, dtype: float64


In [52]:
eth_1h['pct_change'] = calc_pct_change(eth_1h['high'], eth_1h['low'], eth_1h['open'])
eth_1h['hour'] = pd.DatetimeIndex(eth_1h['date']).hour

# Only take volatility from Saturday
eth_1h_saturday = eth_1h[pd.DatetimeIndex(eth_1h['date']).day_name() == 'Saturday']

hourly_volatility = eth_1h.groupby('hour')['pct_change'].mean().sort_values(ascending=False)
print(hourly_volatility)

hour
11    0.015472
19    0.014008
10    0.013859
9     0.013478
18    0.013072
17    0.012644
8     0.012570
15    0.012349
7     0.012239
16    0.011714
5     0.011542
12    0.011506
4     0.011314
3     0.011192
6     0.011139
14    0.010986
20    0.010895
13    0.010425
23    0.010207
22    0.010100
21    0.010055
2     0.010032
0     0.009698
1     0.009190
Name: pct_change, dtype: float64


In [64]:
# Add boolean value to rebalance based on weekday and hour with least volatility

is_weekday = pd.DatetimeIndex(eth_1h['date']).day_name() == 'Saturday'
is_hour = pd.to_datetime(eth_1h['date']).apply(lambda x: x.hour == 1 and x.minute == 0)

rebalance_dates = eth_1h['date'][is_weekday & is_hour]
print(rebalance_dates)

97      2019-01-05 01:00:00
265     2019-01-12 01:00:00
433     2019-01-19 01:00:00
601     2019-01-26 01:00:00
769     2019-02-02 01:00:00
937     2019-02-09 01:00:00
1105    2019-02-16 01:00:00
1273    2019-02-23 01:00:00
1441    2019-03-02 01:00:00
1609    2019-03-09 01:00:00
1770    2019-03-16 01:00:00
1938    2019-03-23 01:00:00
2106    2019-03-30 01:00:00
2274    2019-04-06 01:00:00
2442    2019-04-13 01:00:00
2610    2019-04-20 01:00:00
2778    2019-04-27 01:00:00
2946    2019-05-04 01:00:00
3114    2019-05-11 01:00:00
3272    2019-05-18 01:00:00
3440    2019-05-25 01:00:00
3608    2019-06-01 01:00:00
3776    2019-06-08 01:00:00
3944    2019-06-15 01:00:00
4112    2019-06-22 01:00:00
4280    2019-06-29 01:00:00
4448    2019-07-06 01:00:00
4616    2019-07-13 01:00:00
4784    2019-07-20 01:00:00
4952    2019-07-27 01:00:00
5120    2019-08-03 01:00:00
5288    2019-08-10 01:00:00
5448    2019-08-17 01:00:00
5616    2019-08-24 01:00:00
5784    2019-08-31 01:00:00
5952    2019-09-07 0

## Determine Signals

In [ ]:
from TAcharts.indicators import sma

sma_200 = sma(eth_1h['close'], n=200)
sma_100 = sma(eth_1h['close'], n=100)
sma_50 = sma(eth_1h['close'], n=50)

bullish = (sma_50 > sma_100) & (sma_100 > sma_200)
bearish = (sma_50 < sma_100) & (sma_100 < sma_200)

In [3]:
coins = ['ETH', 'DAI']

wiggle_room = 0.05

slippage = 0.008

initial_capital = 10000

allocations = {
    'bull': [0.75, 0.25],
    'neutral': [0.50, 0.50],
    'bear': [0.25, 0.75]
}

In [6]:
df_prices = pd.read_csv('../data/prices.csv')
df_prices.head()

,date,ETH,BTC,DAI
0,2019-01-01 00:00:00,133.36,3703.56,1
1,2019-01-01 01:00:00,133.56,3713.83,1
2,2019-01-01 02:00:00,134.52,3716.70,1
3,2019-01-01 03:00:00,133.70,3699.95,1
4,2019-01-01 04:00:00,133.86,3713.07,1
